In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import dataloader as dl

In [2]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 500
USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)

다음 기기로 학습합니다: cuda


In [3]:
train_loader=dl.load_trainset("/home/dhk1349/Desktop/Capstone Design2/ntu/xsub/")

In [4]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Generator Code
#nc - number of color channels in the input images. For color images this is 3
#nz - length of latent vector (i.e. size of generator input)
#ngf - relates to the depth of feature maps carried through the generator (Size of feature maps in generator)
#ndf - sets the depth of feature maps propagated through the discriminator (Size of feature maps in discriminator)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

nc=1 #channel이랄 것이 딱히 없음


class DCGANGenerator(nn.Module):
    def __init__(self):
        super(DCGANGenerator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, (2,4), (1,0), bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 16
            
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False, dilation =(1,2)),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 35
            nn.ConvTranspose2d( ngf , 1, 4, 2, (2,0), bias=False, dilation=(1,2)),
            nn.Tanh()
            # state size. (ngf) x 32 x 75
            
        )
    
    
    def forward(self, input):
        return self.main(input)

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            #30*75로 해야겠음
            # input is (nc) x 30 x 75
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 15 x 37
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1,(1, 4), 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [9]:
D = Discriminator().to(DEVICE)
G = DCGANGenerator().to(DEVICE)

d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

In [8]:
criterion=nn.BCELoss()

In [10]:
total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, data in enumerate(train_loader):
        x = np.array(data[0][0])
        y=np.array(data[1])
        X=[]
        for i in range(30):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        X=np.array(X).reshape(1,30,75)
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        
        #print(X.size())
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.tensor(y).to(torch.float32).to(DEVICE)
        fake_labels = torch.tensor([0]).to(torch.float32).to(DEVICE)
        
        # 판별자가 진짜 이미지를 진짜로 인식하는 오차를 예산
        outputs = D(X)[0][0][0]
        
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        #print(real_score)
        # 무작위 텐서로 가짜 이미지 생성
        #z = torch.randn(30,1)
        #z = torch.tensor(np.array(z)).unsqueeze(0).to(DEVICE)
        z=torch.randn(1, nz, 1, 1).to(DEVICE)
        fake_images = torch.reshape(G(z),(1,1,30,75) )
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차를 계산
        outputs = D(fake_images)[0][0][0]
        
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake

        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차를 계산
        fake_images = torch.reshape(G(z),(1,1,30,75) )
        outputs = D(fake_images)[0][0][0]
        g_loss = criterion(outputs, real_labels)
        
        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    print('Epoch [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
          .format(epoch, EPOCHS, d_loss.item(), g_loss.item(), 
                  real_score.mean().item(), fake_score.mean().item()))
    if(epoch%3==0):
        torch.save(G.state_dict(), './snapshot/cnngan/1106_cnn_'+str(epoch)+'.pt')

Epoch [0/500], d_loss: 0.1556, g_loss: 6.1577, D(x): 0.86, D(G(z)): 0.00
Epoch [1/500], d_loss: 0.0390, g_loss: 0.0005, D(x): 0.04, D(G(z)): 0.00
Epoch [2/500], d_loss: 0.0207, g_loss: 0.0012, D(x): 0.02, D(G(z)): 0.00
Epoch [3/500], d_loss: 0.0569, g_loss: 7.3437, D(x): 0.95, D(G(z)): 0.00
Epoch [4/500], d_loss: 0.0051, g_loss: 0.0013, D(x): 0.00, D(G(z)): 0.00
Epoch [5/500], d_loss: 0.0189, g_loss: 0.0011, D(x): 0.02, D(G(z)): 0.00
Epoch [6/500], d_loss: 0.0090, g_loss: 0.0005, D(x): 0.01, D(G(z)): 0.00
Epoch [7/500], d_loss: 0.0170, g_loss: 6.6788, D(x): 0.99, D(G(z)): 0.01
Epoch [8/500], d_loss: 0.0137, g_loss: 7.5257, D(x): 0.99, D(G(z)): 0.00
Epoch [9/500], d_loss: 0.0018, g_loss: 6.6855, D(x): 1.00, D(G(z)): 0.00
Epoch [10/500], d_loss: 0.0013, g_loss: 8.2306, D(x): 1.00, D(G(z)): 0.00
Epoch [11/500], d_loss: 0.0014, g_loss: 7.6161, D(x): 1.00, D(G(z)): 0.00
Epoch [12/500], d_loss: 0.0003, g_loss: 0.0002, D(x): 0.00, D(G(z)): 0.00
Epoch [13/500], d_loss: 0.0006, g_loss: 0.0002, 

Epoch [111/500], d_loss: 0.0000, g_loss: 13.1151, D(x): 1.00, D(G(z)): 0.00
Epoch [112/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [113/500], d_loss: 0.0000, g_loss: 36.3212, D(x): 1.00, D(G(z)): 0.00
Epoch [114/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [115/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [116/500], d_loss: 0.0001, g_loss: 11.1319, D(x): 1.00, D(G(z)): 0.00
Epoch [117/500], d_loss: 0.0000, g_loss: 11.9026, D(x): 1.00, D(G(z)): 0.00
Epoch [118/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [119/500], d_loss: 0.0002, g_loss: 12.9010, D(x): 1.00, D(G(z)): 0.00
Epoch [120/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [121/500], d_loss: 0.0008, g_loss: 10.9306, D(x): 1.00, D(G(z)): 0.00
Epoch [122/500], d_loss: 0.0000, g_loss: 12.5316, D(x): 1.00, D(G(z)): 0.00
Epoch [123/500], d_loss: 0.0001, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [124/500], d

Epoch [220/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [221/500], d_loss: 0.0000, g_loss: 41.7816, D(x): 1.00, D(G(z)): 0.00
Epoch [222/500], d_loss: 0.0000, g_loss: 39.9816, D(x): 1.00, D(G(z)): 0.00
Epoch [223/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [224/500], d_loss: 0.0000, g_loss: 53.9271, D(x): 1.00, D(G(z)): 0.00
Epoch [225/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [226/500], d_loss: 0.0000, g_loss: 53.6450, D(x): 1.00, D(G(z)): 0.00
Epoch [227/500], d_loss: 0.0000, g_loss: 52.8219, D(x): 1.00, D(G(z)): 0.00
Epoch [228/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [229/500], d_loss: 0.0000, g_loss: 53.5927, D(x): 1.00, D(G(z)): 0.00
Epoch [230/500], d_loss: 0.0000, g_loss: 50.3637, D(x): 1.00, D(G(z)): 0.00
Epoch [231/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [232/500], d_loss: 0.0000, g_loss: 51.7104, D(x): 1.00, D(G(z)): 0.00
Epoch [233/500], 

Epoch [329/500], d_loss: 0.0000, g_loss: 66.5204, D(x): 1.00, D(G(z)): 0.00
Epoch [330/500], d_loss: 0.0000, g_loss: 65.7050, D(x): 1.00, D(G(z)): 0.00
Epoch [331/500], d_loss: 0.0000, g_loss: 65.2298, D(x): 1.00, D(G(z)): 0.00
Epoch [332/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [333/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [334/500], d_loss: 0.0000, g_loss: 65.1916, D(x): 1.00, D(G(z)): 0.00
Epoch [335/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [336/500], d_loss: 0.0000, g_loss: 65.3682, D(x): 1.00, D(G(z)): 0.00
Epoch [337/500], d_loss: 0.0000, g_loss: 64.7872, D(x): 1.00, D(G(z)): 0.00
Epoch [338/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [339/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [340/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [341/500], d_loss: 0.0004, g_loss: 66.1693, D(x): 1.00, D(G(z)): 0.00
Epoch [342/500], d

Epoch [438/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [439/500], d_loss: 0.0000, g_loss: 55.8226, D(x): 1.00, D(G(z)): 0.00
Epoch [440/500], d_loss: 0.0000, g_loss: 51.8153, D(x): 1.00, D(G(z)): 0.00
Epoch [441/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [442/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [443/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [444/500], d_loss: 0.0000, g_loss: 50.8683, D(x): 1.00, D(G(z)): 0.00
Epoch [445/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [446/500], d_loss: 0.0000, g_loss: 51.4576, D(x): 1.00, D(G(z)): 0.00
Epoch [447/500], d_loss: 0.0000, g_loss: 53.3680, D(x): 1.00, D(G(z)): 0.00
Epoch [448/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [449/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [450/500], d_loss: 0.0000, g_loss: 0.0000, D(x): 0.00, D(G(z)): 0.00
Epoch [451/500], d_l

In [ ]:
total_step = len(train_loader)
EPOCH=500
for epoch in range(100, 501):
    for i, data in enumerate(train_loader):
        x = np.array(data[0][0])
        
        X=[]
        for i in range(30):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        X=np.array(X).reshape(1,30,75)
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        
        #print(X.size())
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.tensor([1]).to(torch.float32).to(DEVICE)
        fake_labels = torch.tensor([1]).to(torch.float32).to(DEVICE)
        
        # 판별자가 진짜 이미지를 진짜로 인식하는 오차를 예산
        outputs = D(X)[0][0][0]
        
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        #print(real_score)
        # 무작위 텐서로 가짜 이미지 생성
        #z = torch.randn(30,1)
        #z = torch.tensor(np.array(z)).unsqueeze(0).to(DEVICE)
        z=torch.randn(1, nz, 1, 1).to(DEVICE)
        fake_images = torch.reshape(G(z),(1,1,30,75) )
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차를 계산
        outputs = D(fake_images)[0][0][0]
        
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake
        
        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차를 계산
        fake_images = torch.reshape(G(z),(1,1,30,75) )
        outputs = D(fake_images)[0][0][0]
        g_loss = criterion(outputs, real_labels)
        
        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    print('Epoch [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
          .format(epoch, EPOCHS, d_loss.item(), g_loss.item(), 
                  real_score.mean().item(), fake_score.mean().item()))
    if(epoch%3==0):
        torch.save(G.state_dict(), './snapshot/1106_cnn_'+str(epoch)+'.pt')

In [ ]:
output.size()

In [ ]:
np.save("./snapshot/gan_cnn_generated_action99.npy", output.detach().numpy().reshape(1,30,75))

In [ ]:
a=1
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.main = nn.Sequential(
            #30*75로 해야겠음
            # input is (nc) x 30 x 75
            nn.Conv2d(a, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 15 x 37
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1,(1, 4), 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
test_input=torch.randn(1, 1, 30, 75)

In [ ]:
testmodel=test()

In [ ]:
out=testmodel(test_input)

In [ ]:
out